In [1]:
import re
import requests
from selenium import webdriver

In [2]:
url='https://kyfw.12306.cn/otn/resources/js/framework/station_name.js?station_version=1.9260'
response=requests.get(url)
station_text=response.text # 获取响应内容
code_list=re.split('\W+', station_text)#split the string by non-letters

In [3]:
def find_code(city_name):
    station_code=code_list[code_list.index(city_name)+1]#find city name's index, the code in behind the name
    return station_code

In [4]:
find_code('重庆')

'cqi'

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

url='https://12306.cn/'
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('ignore-certificate-errors')

driver = webdriver.Chrome(r'C:\Users\Administrator\Documents\meijmPy\chromedriver.exe',options=options)
driver.get(url)

In [6]:
def element_input(ele_id,ele_input):
    element=driver.find_element(By.ID, ele_id)
    try:#search has no input
        element.clear()
        element.click()
        element.send_keys(ele_input)
    except:
        print(ele_id+' no input')
    element.send_keys(Keys.ENTER)
    time.sleep(1)

In [7]:
element_input('fromStationText','上海')
element_input('toStationText','重庆')
element_input('train_date','2023-05-07')
element_input('search_one','')

search_one no input


In [8]:
#drive switch 
print(driver.window_handles)
print(driver.current_window_handle)
driver.switch_to.window(driver.window_handles[1])#switch to new tab
print(driver.current_url)

['CDwindow-A1685171F35B86AE737655DEE31A9105', 'CDwindow-BE6AB2468391AAF87F22F8011E6C57E5']
CDwindow-A1685171F35B86AE737655DEE31A9105
https://kyfw.12306.cn/otn/leftTicket/init?linktypeid=dc&fs=%E4%B8%8A%E6%B5%B7,SHH&ts=%E9%87%8D%E5%BA%86%E5%8C%97,CUW&date=2023-05-07&flag=N,N,Y


In [9]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

page_text = driver.page_source
soup = BeautifulSoup(page_text, 'html.parser')

In [10]:
find_id = soup.find(id='queryLeftTable')
columlist=['出发站','到达站','出发时间','达到时间','时长','商务座','一等座','二等座','高级软卧','软卧','动卧','硬卧','软座','硬座','无座','其他','预定']
train_df = pd.DataFrame([],columns=columlist)#to store all info
#each train info， and all train' id start with ticket in source page
tr_list=find_id.find_all(name='tr',id=re.compile("ticket_*"))
for train in tr_list:
    td_list=train.find_all('td')#each seat info
    count=0#we need to extract the train info from the first td
    seat_list=[]#to store seat info
    for treat in td_list:
        if count==0:
            train_id=treat.find(title='点击查看停靠站信息')
            print(train_id.text)
            for strong in treat.find_all('strong'):#the time and station info in strong
                print(strong.text)
                seat_list.append(strong.text)
        else:
            print(treat.text)
            seat_list.append(treat.text)
        count=count+1
    train_df1=pd.DataFrame(seat_list,index=columlist,columns=[train_id.text]).T
    train_df=pd.concat([train_df,train_df1],axis=0)
                        

D2206
上海虹桥
重庆北
06:01
19:17
13:16
--
有
有
--
--
--
--
--
--
有
--
预订
D352
上海虹桥
重庆北
06:13
18:07
11:54
--
有
有
--
--
--
--
--
--
有
--
预订
D636
上海虹桥
重庆北
06:32
18:38
12:06
--
有
有
--
--
--
--
--
--
有
--
预订
D3072
上海虹桥
重庆北
07:02
19:24
12:22
--
有
有
--
--
--
--
--
--
有
--
预订
G1974
上海虹桥
重庆西
07:17
20:27
13:10
4
有
有
--
--
--
--
--
--
--
--
预订
D3056
上海
重庆北
07:23
19:38
12:15
--
有
有
--
--
--
--
--
--
有
--
预订
D2216
上海虹桥
重庆北
07:37
21:20
13:43
--
有
有
--
--
--
--
--
--
有
--
预订
D2212
上海虹桥
重庆北
07:42
20:19
12:37
--
18
有
--
--
--
--
--
--
无
--
预订
G3288
上海虹桥
沙坪坝
07:49
21:48
13:59
5
10
有
--
--
--
--
--
--
--
--
预订
D2268
上海虹桥
重庆北
08:01
21:30
13:29
--
有
有
--
--
--
--
--
--
有
--
预订
G1337
上海虹桥
重庆西
08:22
19:46
11:24
15
有
有
--
--
--
--
--
--
--
--
预订
D952
上海
重庆北
08:30
19:00
10:30
--
有
有
--
--
--
--
--
--
有
--
预订
D956
上海
重庆北
09:30
20:14
10:44
--
有
有
--
--
--
--
--
--
有
--
预订
G1476
上海虹桥
重庆北
09:51
20:32
10:41
候补
候补
有
--
--
--
--
--
--
--
--
预订
G1333
上海虹桥
重庆西
11:44
23:09
11:25
候补
候补
有
--
--
--
--
--
--
--
--
预订
G1532
上海虹桥
重庆

In [11]:
train_df

,出发站,到达站,出发时间,达到时间,时长,商务座,一等座,二等座,高级软卧,软卧,动卧,硬卧,软座,硬座,无座,其他,预定
D2206,上海虹桥,重庆北,06:01,19:17,13:16,--,有,有,--,--,--,--,--,--,有,--,预订
D352,上海虹桥,重庆北,06:13,18:07,11:54,--,有,有,--,--,--,--,--,--,有,--,预订
D636,上海虹桥,重庆北,06:32,18:38,12:06,--,有,有,--,--,--,--,--,--,有,--,预订
D3072,上海虹桥,重庆北,07:02,19:24,12:22,--,有,有,--,--,--,--,--,--,有,--,预订
G1974,上海虹桥,重庆西,07:17,20:27,13:10,4,有,有,--,--,--,--,--,--,--,--,预订
D3056,上海,重庆北,07:23,19:38,12:15,--,有,有,--,--,--,--,--,--,有,--,预订
D2216,上海虹桥,重庆北,07:37,21:20,13:43,--,有,有,--,--,--,--,--,--,有,--,预订
D2212,上海虹桥,重庆北,07:42,20:19,12:37,--,18,有,--,--,--,--,--,--,无,--,预订
G3288,上海虹桥,沙坪坝,07:49,21:48,13:59,5,10,有,--,--,--,--,--,--,--,--,预订
D2268,上海虹桥,重庆北,08:01,21:30,13:29,--,有,有,--,--,--,--,--,--,有,--,预订


In [406]:
columlist=['出发站','到达站','出发时间','达到时间','时长','商务座','一等座','二等座','高级软卧','软卧','动卧','硬卧','软座','硬座','无座','其他','预定']
train_df = pd.DataFrame([],columns=columlist)#to store all info

In [362]:
columlist1=['出发站','到达站','出发时间','达到时间','时长']
columlist2=['商务座','一等座','二等座','高级软卧','软卧','动卧','硬卧','软座','硬座','无座','其他']
init_np2=np.zeros((len(tr_list),len(columlist2)),dtype=int)-1
df2 = pd.DataFrame(init_np2,index=trainid_list, columns=columlist2)
infolist=[['unknown']*len(columlist1)]*len(tr_list)
df1 = pd.DataFrame(data=infolist,index=trainid_list, columns=columlist1)
df=pd.concat([df1, df2],axis=1)

In [371]:
s1 = pd.Series([])
s2 = pd.Series(['c', 'd'])
s3 = pd.Series(['c', 'd'])
pd.concat([s2, s3])

0    c
1    d
0    c
1    d
dtype: object

In [325]:
trainid_list

['D2206',
 'D352',
 'D636',
 'D3072',
 'G1974',
 'D3056',
 'D2216',
 'D2212',
 'G3288',
 'D2268',
 'G1337',
 'D952',
 'D956',
 'G1476',
 'G1333',
 'G1532',
 'K71',
 'Z257']

In [348]:
infolist=[['unknown']*len(columlist1)]*len(tr_list)

In [353]:
df1 = pd.DataFrame(data=infolist,index=trainid_list, columns=columlist1)

In [358]:
df=pd.concat([df1, df2],axis=1)

In [359]:
df

,出发站,到达站,出发时间,达到时间,时长,商务座,一等座,二等座,高级软卧,软卧,动卧,硬卧,软座,硬座,无座,其他
D2206,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D352,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D636,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D3072,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G1974,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D3056,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D2216,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D2212,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
G3288,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
D2268,unknown,unknown,unknown,unknown,unknown,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [286]:
train_id_id=train_id['id']

In [287]:
element=driver.find_element(By.ID, train_id_id)

In [289]:
element.click()

In [266]:
for tt in treat.find_all('strong'):
    print(tt.text)

上海南
重庆北
19:07
12:44
17:37


In [300]:
np.zeros(5,4)

TypeError: data type not understood

In [149]:
find_id['id']

'queryLeftTable'

In [150]:
find_id.name

'tbody'

In [151]:
find_id.attrs

{'id': 'queryLeftTable'}

In [161]:
find_id['id']

'queryLeftTable'

In [169]:
find_id.get_attribute_list('id')

['queryLeftTable']

In [171]:
find_id

<tbody id="queryLeftTable"><tr class="bgc" id="ticket_5l000D220602_01_17"><td colspan="4" width="370"><div class="ticket-info clearfix" id="train_num_0"><div class="train" id="ticket_5l000D220602_01_17_train"><div><a class="number" href="javascript:" id="5l000D220602_AOH_CUW" onclick="myStopStation.open('0','5l000D220602','AOH','CUW','20230507','3');" title="点击查看停靠站信息">D2206</a><div class="train-type"></div></div><span class="lookup" id="5l000D220602_01_17_C51435XW2CdKbQS2DdpLIkQbjr%2BC9UvLvOYujPMMNz%2FSdMGY_MOO" onclick="showTicketPrice(this)"><span style="display:none;">查看票价</span><b aria-label="查看票价,按回车键操作" tabindex="0" title="查看票价"></b></span></div><div class="cdz"><strong class="start-s" title="上海虹桥">上海虹桥</strong><strong>重庆北</strong></div><div class="cds"><strong class="start-t">06:01</strong><strong class="color999">19:17</strong></div><div class="ls" id="5l000D220602_ls"><strong>13:16</strong><span>当日到达</span></div></div></td> <td align="center" hbdata="2023-05-07#5l000D220602#T

In [177]:
find_id.contents[0].text

'D2206查看票价上海虹桥重庆北06:0119:1713:16当日到达 --有有 -- -- -- -- -- --有 -- 预订'

In [188]:
find_id.contents[0].contents[0].text

'D2206查看票价上海虹桥重庆北06:0119:1713:16当日到达'

In [191]:
find_id.contents[0].contents[1].text

AttributeError: 'NavigableString' object has no attribute 'text'

In [192]:
find_id.contents[0].contents[2].text

'--'

In [193]:
find_id.contents[0].contents[3].text

'有'

In [180]:
find_id.contents[1].text

''

In [181]:
find_id.contents[2].text

'D352查看票价上海虹桥重庆北06:1318:0711:54当日到达 --有有 -- -- -- -- -- --有 -- 预订'

In [182]:
find_id.contents[3].text

''

In [206]:
a=0
for child in find_id.children:
    print(child)
    a=a+1
    if a>2:
        break

<tr class="bgc" id="ticket_5l000D220602_01_17"><td colspan="4" width="370"><div class="ticket-info clearfix" id="train_num_0"><div class="train" id="ticket_5l000D220602_01_17_train"><div><a class="number" href="javascript:" id="5l000D220602_AOH_CUW" onclick="myStopStation.open('0','5l000D220602','AOH','CUW','20230507','3');" title="点击查看停靠站信息">D2206</a><div class="train-type"></div></div><span class="lookup" id="5l000D220602_01_17_C51435XW2CdKbQS2DdpLIkQbjr%2BC9UvLvOYujPMMNz%2FSdMGY_MOO" onclick="showTicketPrice(this)"><span style="display:none;">查看票价</span><b aria-label="查看票价,按回车键操作" tabindex="0" title="查看票价"></b></span></div><div class="cdz"><strong class="start-s" title="上海虹桥">上海虹桥</strong><strong>重庆北</strong></div><div class="cds"><strong class="start-t">06:01</strong><strong class="color999">19:17</strong></div><div class="ls" id="5l000D220602_ls"><strong>13:16</strong><span>当日到达</span></div></div></td> <td align="center" hbdata="2023-05-07#5l000D220602#TZ_#" hbid="2023-05-07#AOH#C

In [200]:
len(list(find_id.children))

37

In [210]:
child.text

'D352查看票价上海虹桥重庆北06:1318:0711:54当日到达 --有有 -- -- -- -- -- --有 -- 预订'

In [211]:
child.string

In [224]:
tr_list=find_id.find_all('tr')

In [226]:
tr_list[0].text

'D2206查看票价上海虹桥重庆北06:0119:1713:16当日到达 --有有 -- -- -- -- -- --有 -- 预订'

In [227]:
tr_list[1].text

''

In [228]:
tr_list[2].text

'D352查看票价上海虹桥重庆北06:1318:0711:54当日到达 --有有 -- -- -- -- -- --有 -- 预订'

In [232]:
grandson=child.find_all('td')

In [234]:
grandson[0].text

'D3056查看票价上海重庆北07:2319:3812:15当日到达'

In [235]:
grandson[1].text

'--'

In [236]:
grandson[2].text

'有'